# Cleaning the Heart Disease Dataset from kaggle
This dataset is for 2014 and provides the heart disease mortality rates for different
ethnicities within the counties across the US.

In [23]:
# Dependencies
import os
import csv
import pandas as pd

In [24]:
# Import the dataset
file = "Data/Heart_Disease_Mortality_Data_Among_US_Adults_35_by_State_Territory_and_County.csv"
df = pd.read_csv(file)

In [25]:
# Deleting Useless Columns
df = df.drop(columns= ["Year", "GeographicLevel","DataSource","Class",'Topic',"Data_Value_Type",
                       'Data_Value_Footnote_Symbol','Data_Value_Footnote','StratificationCategory1',
                       'StratificationCategory2','TopicID','LocationID','Data_Value_Unit'])
df = df.rename(columns={"LocationAbbr": "State", "LocationDesc": "County", 
                        "Data_Value": "Value", "Stratification1": "Gender", 
                        "Stratification2": "Race/Ethnicity", "Location 1": "LatLng"})

# Dropping Rows that are missing data
df = df.dropna(how='any')

df.to_csv("heart_partialclean.csv")


In [6]:
# Separate the latitude and longitude columns
# We will use the latitude and longitude data for locations and google API
split = df["LatLng"].str.split("(", n = 1, expand = True) 
split = split[1].str.split(", ", n = 1, expand = True) 
splitLng = split[1].str.split(")", n = 1, expand = True) 

# making seperate first name column from new data frame 
df["Lat"]= split[0] 
  
# making seperate last name column from new data frame 
df["Lng"]= splitLng[0] 
  
# Dropping old Name columns 
df.drop(columns =["LatLng"], inplace = True) 


In [ ]:
# Sorting by county
county = pd.DataFrame(df)
county = county.loc[county['Gender']== "Overall"]
county = county.loc[county['Race/Ethnicity'] == "Overall"]

# Reorder by rate of heart disease
county = county.sort_values(['Value'],ascending=False)
#.groupby(['State', 'County'])

county.head()


In [27]:
# Sorting by Ethnicity (White)
white = pd.DataFrame(df)
white = white.loc[white['Gender']== "Overall"]
white = white.loc[white['Race/Ethnicity'] == "White"]
white = white.sort_values(['State'])


In [28]:
# Sorting by Ethnicity (Black)
black = pd.DataFrame(df)
black = black.loc[black['Gender']== "Overall"]
black = black.loc[black['Race/Ethnicity'] == "Black"]
black = black.sort_values(['State'])


In [9]:
# Sorting by Ethnicity (Hispanic)
hispanic = pd.DataFrame(df)
hispanic = hispanic.loc[hispanic['Gender']== "Overall"]
hispanic = hispanic.loc[hispanic['Race/Ethnicity'] == "Hispanic"]
hispanic = hispanic.sort_values(['State'])


In [10]:
# Sorting by Ethnicity (American Indian)
Indian = pd.DataFrame(df)
Indian = Indian.loc[Indian['Gender']== "Overall"]
Indian = Indian.loc[Indian['Race/Ethnicity'] == "American Indian and Alaskan Native"]
Indian = Indian.sort_values(['State'])


In [11]:
# Sorting by Ethnicity (Asian)
Asian = pd.DataFrame(df)
Asian = Asian.loc[Asian['Gender']== "Overall"]
Asian = Asian.loc[Asian['Race/Ethnicity'] == "Asian and Pacific Islander"]
Asian = Asian.sort_values(['State'])


In [ ]:
# Time to save all the things
county.to_csv("countydata.csv")
white.to_csv("whitedata.csv")
black.to_csv("blackdata.csv")
hispanic.to_csv("hispanicdata.csv")
Indian.to_csv("indiandata.csv")
Asian.to_csv("asiandata.csv")

### Column Names
#### State = State (self explanatory))
#### County = County (self explanatory)
#### Value = heart disease mortality rate.
#### Gender = male, female, or overall = combing all genders together
#### Race/Ethnicity = This provides the overall mortality rate ACROSS all ethnicities within a county and proviedes a weighted average of overall heart disease mortality. It also breaks it down by different ethnic groups
#### Lat = Latitude of county
#### Lng = Longitude of county

# Pulling Census data
We will need data by county for populations of different ethnic groups

See: https://github.com/CommerceDataService/census-wrapper for library documentation

See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

In [1]:
# Dependencies that we may or may not need eventually
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
import time
import pandas as pd

In [ ]:
# Census & gmaps API Keys
# Bring your own API keys!
census_api = input("What's your Census API key?")
gkey = input ("What's your Gmaps API key?")
c = Census(census_api, year=2014)

In [15]:
# From the documentation noted above, we can pull these variable names and get the data for each county in the US
census_data_pop = c.acs5.get(("NAME", "B02001_002E", "B02001_003E", "B02001_004E",
                              "B02001_005E","B02001_006E", "B02001_008E", 
                              "B03001_003E", "B01003_001E"),{'for': 'county:*'})

# Convert to DataFrame
census_pop_pd = pd.DataFrame(census_data_pop)

# Column renaming
census_pop_pd = census_pop_pd.rename(columns={"B02001_002E": "whitetot",
                                              "B02001_003E": "blacktot", 
                                              "B02001_004E": "amerindtot",
                                              "B02001_005E": "asiantot",
                                              "B02001_006E": "nathawtot", 
                                              "B02001_008E": "multitot", 
                                              "B03001_003E": "hisptot",
                                              "B01003_001E" : "totalpop",
                                              "NAME": "Name", "county": "Country"})
# We'll manipulate the data a little now
# We know that the data from the heart disease mortality dataset is "per 100,000"
# Therefore we want to convert our population data into the same metric

# finds a conversion factor for each county
census_pop_pd["factor"]= census_pop_pd["totalpop"]/100000

# Creates a per 100,000 population value for each racial/ethnic group
census_pop_pd["white_perh"] = census_pop_pd["whitetot"] / census_pop_pd["factor"]
census_pop_pd["black_perh"] = census_pop_pd["blacktot"] / census_pop_pd["factor"]
census_pop_pd["amerind_perh"] = census_pop_pd["amerindtot"] / census_pop_pd["factor"]
census_pop_pd["asian_perh"] = census_pop_pd["asiantot"] / census_pop_pd["factor"]
census_pop_pd["nathaw_perh"] = census_pop_pd["nathawtot"] / census_pop_pd["factor"]
census_pop_pd["multi_perh"] = census_pop_pd["multitot"] / census_pop_pd["factor"]
census_pop_pd["hisp_perh"] = census_pop_pd["hisptot"] / census_pop_pd["factor"]

In [ ]:
# Unfortunately the data location is in a "County, State" format
# So we have to take WAY too much time and split that column. 
# And then we have to get rid of a space before the state name.
# And then we have to match the state name to its abbreviation so we can match it later. 
# IF you were paying attention, a similar conversion was done above in the mortality data. 
# That conversion seemed to be much simpler than what we did here. So many ways to skin the data.

# Import a csv file with a column of state names and their respective abbreviations
states = pd.read_csv("state_abbreviations.csv")

# Make some space for the incoming data
census_pop_pd["CountyName"] = ""
census_pop_pd["StateName"] = "" 
census_pop_pd["State"] = ""

for i in range(0,len(census_pop_pd)):
    st = census_pop_pd.Name[i].split(",")
    census_pop_pd.CountyName[i] = st[0]
    ct = st[1].split(" ")
    census_pop_pd.StateName[i] = ct[1]
    for j in range(0, 50):
        if census_pop_pd.StateName[i] == states.state[j]:
            census_pop_pd.State[i] = states.abbreviation[j]   


In [17]:
# Add in Poverty Rate (Poverty Count / Population)
# This will perhaps be used in an analysis... better too much data than not enough
census_pop_pd["whiteperc"] = 100*(census_pop_pd["whitetot"].astype(int)/census_pop_pd["totalpop"].astype(int))
census_pop_pd["blackperc"] = 100*(census_pop_pd["blacktot"].astype(int)/census_pop_pd["totalpop"].astype(int))
census_pop_pd["amerindperc"] = 100*(census_pop_pd["amerindtot"].astype(int)/census_pop_pd["totalpop"].astype(int))
census_pop_pd["asianperc"] = 100*(census_pop_pd["asiantot"].astype(int)/census_pop_pd["totalpop"].astype(int))
census_pop_pd["nathawperc"] = 100*(census_pop_pd["nathawtot"].astype(int)/census_pop_pd["totalpop"].astype(int))
census_pop_pd["multiperc"] = 100*(census_pop_pd["multitot"].astype(int)/census_pop_pd["totalpop"].astype(int))
census_pop_pd["hispperc"] = 100*(census_pop_pd["hisptot"].astype(int)/census_pop_pd["totalpop"].astype(int))


In [ ]:
# Drop some useless columns
census_pop_pd = census_pop_pd.drop(columns= ["StateName", "Country", "Unnamed: 0", "state", "Name"]]

In [ ]:
# Saving this Data!
census_pop_pd.to_csv("CensusTotalPopulation_perH.csv")

### Column Names
#### totalpop = total population of each county
#### whitetot = White population of each county
#### blacktot = Black population of each county
#### amerindtot = American Indian population of each county
#### asiantot = Asian population of each county
#### nathawtot = Native Hawaiian population of each county
#### multitot= multi-racial population of each county
#### hisptot = Hispanic population of each county
#### factor = the factor needed to get the population data into per 100,000
#### white_perh = White population per 100,000 
#### black_perh = Black population per 100,000 
#### amerind_perh = American Indian population per 100,000 
#### asian_perh = Asian population population per 100,000 
#### nathaw_perh = Native Hawaiian population per 100,000 
#### multi_perh = multi-racial population per 100,000 
#### hisp_perh = Hispanic population per 100,000 
#### CountyName = Name of the county, separated from state
#### State = state!
#### whiteperc = White population percentage 
#### blackperc = Black population percentage
#### amerindperc = American Indian population percentage
#### asianperc = Asian population percentage
#### nathawperc = Native Hawaiian population percentage
#### multiperc = multi-racial population percentage
#### hispperc = Hispanic population percentage

# Google API for Fast Food Data Acquisition

In [ ]:
# Import heart disease DataFrame with correct google Lat/Lat format
google_Latlng = pd.read_csv("Data/heart_partialclean.csv")

# Splitting Latitude and Longitude apart again 
# (can you tell these were done by different team members?)
split = google_Latlng["LatLng"].str.split("(", n = 1, expand = True) 
split = split[1].str.split(")", n = 1, expand = True) 
google_Latlng["Lat/Lng"]= split[0] 
  
# Dropping old columns 
google_Latlng.drop(columns =["LatLng"], inplace = True) 
google_Latlng.count()

# Cleaning up google_Latlng df
google_Latlng = google_Latlng.loc[google_Latlng['Gender']== "Overall"]
google_Latlng = google_Latlng.loc[google_Latlng['Race/Ethnicity'] == "Overall"]
google_Latlng = google_Latlng.sort_values(['Value'],ascending=False)


In [ ]:
# Creating csv for fast food counts for all counties
total_fast_food_count = []

# A goog place to start
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?query=Fast+Food&"
token = 0
if token < 1:
    for i in google_Latlng["Lat/Lng"]:
        params = {
        "location": i,
        "keyword": "fast",
        "radius": 35000,
        "type": "restaurant",
        "key": gkey}
        response = requests.get(base_url, params=params)

        # convert response to json
        places_data = response.json()
        # Print the json (pretty printed)
        # print(json.dumps(places_data, indent=4, sort_keys=True))
        
        total_fast_food_count.append(len(places_data["results"]))

In [ ]:
# Combine the DataFrames together
total_FF = google_Latlng
total_FF["Fast Food Count"] = total_fast_food_count


In [29]:
# Save the Data!
total_FF.to_csv("TotalFF.csv", index=False)

### Column Names 
###### (all self explanatory we hope!)
#### State
#### County
#### Value
#### Gender
#### Rave/Ethnicity
#### Lat
#### Lng
#### Fast Food Count

# Google API for Hospital Data Acquisition

In [ ]:
# Things you'll need!
import os
import csv
import pandas as pd
import json
import requests

# Importing and reading csv
file = "countydata.csv"
county_df = pd.read_csv(file)

# Base URL needed to make the API call
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

target_type = "hospital"

# Just in case youre running just this section or want to use a different key :)
gkey = input("Enter an API key. ")

In [ ]:
# Create some places to hold the incoming data
rating = []
rating_count = []
hospital_name = []
county_name = []
state_name = []

# Start looping!
for index, row in county_df.iterrows():
    target_county = row["County"]
    target_state = row["State"]
    params = {
        "query": f"{target_county}, {target_state}",
        "type": target_type,
        "key": gkey}
    
    response = requests.get(base_url, params=params).json()
    
    for i in range(len(response["results"])):
        rating.append(response["results"][i]["rating"])
        rating_count.append(response["results"][i]["user_ratings_total"])
        hospital_name.append(response["results"][i]["name"])
        county_name.append(row["County"])
        state_name.append(row["State"])
        

In [ ]:
# Create a dictionary of all the data acquired above
data = {"County": county_name, "State": state_name, "Hospital Name": hospital_name, 
        "Rating": rating, "Rating Count": rating_count}

# Turn the dictionary into a dataframe
hospital_data_df = pd.DataFrame(data, columns=["County", "State", "Hospital Name", "Rating", "Rating Count"])

### Column Names
#### County
#### State
#### Hospital Name
#### Rating = How much people like this hospital. 
#### Rating Count = Number of ratings used to go into the calculation of a rating

In [ ]:
# Save the Data
hospital_data_df.to_csv(r"cleaned_hospital_data.csv")

In [ ]:
# Now that the data is saved from the Hospital API, we want to do some counting
# Drop some columns cuz we don't need them righ tnow
columns = ["List_ID", "Hospital Name", "Rating", "Rating Count"]
hospital_data_df.drop(columns, axis=1, inplace=True)

In [ ]:
# Create a full location name for each row
hospital_data_df["Full Location"] = hospital_data_df["County"] + " " + hospital_data_df["State"]


In [ ]:
# Now we want to see how many different "Full Locations" names we have
result = hospital_data_df.groupby("Full Location").first()

result["Hospital Count"] = hospital_data_df["Full Location"].value_counts()
result.reset_index(inplace=True)


In [ ]:
# Then drop the "Full Location" column because we wont need it in ANY of the analyses
result = result.drop("Full Location", axis=1)

In [ ]:
# Save the Data
result.to_csv(r"number_of_hospitals_per_county.csv")

### Column Names
#### County
#### State
#### Hospital count = number of hospitals located in each county (within a 21 mile radius that is)